In [23]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

# CSV 파일 경로
train_csv_path = './imoticon/train.csv'
test_csv_path = './imoticon/test.csv'

# 이미지 저장 경로
train_img_dir = 'target_train_images'
test_img_dir = 'target_test_images'

# 디렉토리 생성 함수
def make_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# 이미지 저장 함수
def save_images_from_csv(csv_path, img_dir, is_train=True):
    df = pd.read_csv(csv_path)
    
    # 필요한 디렉토리 생성
    if is_train:
        labels = df['label'].unique()
        for label in labels:
            make_dir(os.path.join(img_dir, label))
    else:
        make_dir(img_dir)
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        img_array = np.array(row.iloc[2:].values if is_train else row.iloc[1:].values, dtype=np.uint8).reshape(32, 32)
        img = Image.fromarray(img_array, mode='L')  # 'L' for grayscale
        
        if is_train:
            label = row['label']
            img_path = os.path.join(img_dir, label, f"{row['ID']}.png")
        else:
            img_path = os.path.join(img_dir, f"{row['ID']}.png")
        
        img.save(img_path)

# 실행
make_dir(train_img_dir)
make_dir(test_img_dir)

save_images_from_csv(train_csv_path, train_img_dir, is_train=True)
save_images_from_csv(test_csv_path, test_img_dir, is_train=False)


100%|██████████| 250/250 [00:00<00:00, 561.52it/s]


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 원본 데이터셋 폴더 경로
dataset_folder = "/content/target_train_images"

# 이동할 폴더 경로
train_folder = "/content/train"
val_folder = "/content/valid"

# 클래스 폴더 목록 생성
class_folders = [folder for folder in os.listdir(dataset_folder) if os.path.isdir(os.path.join(dataset_folder, folder))]

# 각 클래스 폴더의 데이터 이동
for class_folder in class_folders:
    class_folder_path = os.path.join(dataset_folder, class_folder)

    # 클래스 폴더 안의 파일 목록 가져오기
    file_list = os.listdir(class_folder_path)

    # 훈련 및 검증 데이터로 나누기
    train_files, val_files = train_test_split(file_list, test_size=0.2, random_state=2025)

    # 훈련 데이터를 train 폴더로 이동
    for file_name in train_files:
        src = os.path.join(class_folder_path, file_name)
        dst = os.path.join(train_folder, class_folder, file_name)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)

    # 검증 데이터를 val 폴더로 이동
    for file_name in val_files:
        src = os.path.join(class_folder_path, file_name)
        dst = os.path.join(val_folder, class_folder, file_name)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)
